# Multilingual prediction

In [19]:
# dependencies
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

In [20]:
# data
df_urdu = pd.read_csv(r'..\Data\Urdu\features_urdu.csv')
df_english = pd.read_csv(r'..\Data\English\features_english.csv')
df_german = pd.read_csv(r'..\Data\German\features_german.csv')
df_italian = pd.read_csv(r'..\Data\Italian\features_italian.csv')

df_engger = pd.concat([df_english, df_german], ignore_index=True)
df_engita = pd.concat([df_english, df_italian], ignore_index=True)
df_gerita = pd.concat([df_german, df_italian], ignore_index=True)

df_western = pd.concat([df_english, df_german, df_italian], ignore_index=True)

combined_languages = {
    'english_and_german': df_engger,
    'english_and_italian': df_engita,
    'german_and_italian': df_gerita,
    'western': df_western
}

Define Functions for better overview and easier loops

In [21]:
def separate_feats_labs(df_train, df_test):
    X_train = df_train.drop(["emotion", "speaker_id", "filename", "valence"], axis=1) # Features
    y_train = df_train['valence']

    X_test = df_test.drop(["emotion", "speaker_id", "filename", "valence"], axis=1) # Features
    y_test = df_test['valence']
    return X_train, y_train, X_test, y_test

def compare_models(models: dict, X_train, y_train, X_test, y_test, results):
    
    for name, model in models.items():
    
        # Define pipeline: Scaling + Model
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model)
            ])
   
        # Fit the model on training data
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
    
        # Metrics calculation
        accuracy = accuracy_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
    
        # Save results
        results.append({
            'Model': name,
            'Accuracy': accuracy,
            'Balanced Accuracy': balanced_accuracy,
            'Precision (Weighted)': precision,
            'Recall (Weighted)': recall,
            'F1-Score (Weighted)': f1
        })

    return results

def add_random_dummy(y_train, y_test, results):
    # Generate random predictions from the existing labels
    y_random = np.random.choice(y_train.unique(), size=len(y_test), replace=True)

    # Metrics for Random Baseline
    random_accuracy = accuracy_score(y_test, y_random)
    random_balanced_accuracy = balanced_accuracy_score(y_test, y_random)
    random_precision = precision_score(y_test, y_random, average='weighted', zero_division=0)
    random_recall = recall_score(y_test, y_random, average='weighted')
    random_f1 = f1_score(y_test, y_random, average='weighted')

    # Save Random Baseline results
    results.append({
        'Model': 'stratified Dummy',
        'Accuracy': random_accuracy,
        'Balanced Accuracy': random_balanced_accuracy,
        'Precision (Weighted)': random_precision,
        'Recall (Weighted)': random_recall,
        'F1-Score (Weighted)': random_f1
    })

    return results

In [22]:
# Define models for comparison
models = {
    'SVM (linear)': SVC(kernel='linear', C=1.0, random_state=42, probability=True),
    'SVM (rbf)': SVC(kernel='rbf', C=1.0, random_state=42, probability=True),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
}

In [23]:
def train_with_test_data(df_train, df_test, western_name):
    # Separate features and labels
    X_train, y_train, X_test, y_test = separate_feats_labs(df_train, df_test)

    # Save results
    results = []

    # Compare different models
    results = compare_models(models, X_train, y_train, X_test, y_test, results)

    # add random baseline/dummy
    results = add_random_dummy(y_train, y_test, results)

    # convert results to dataframe
    results_df = pd.DataFrame(results)

    file_path = f"../Evaluation/Multilingual/multilingual_{western_name}.csv"

    # save the data as csv
    results_df.to_csv(file_path, index=False)

In [24]:
for language, data in combined_languages.items():
    train_with_test_data(df_train=data, df_test=df_urdu, western_name=language)

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\core.py:158: UserWarning: [22:29:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\core.py:158: UserWarning: [22:29:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\core.py:158: UserWarnin